# Lab 01 - Parte 2 - Ranking e Modelo Vetorial (REPOSIÇÃO)

Importando as bibliotecas necessárias

In [1]:
#coding: utf-8
import pandas
import nltk
import numpy as np
import math
__author__ = "Thierry Barros"

indice_invertido = {}
dados_TF = {}

Construindo o índice invertido

função que checa se a palavra já existe no indice invertido, se sim adiciona o id do texto que possui a palavra. Se não cria a chave para palavra e adiciona o id do texto.


In [2]:
def preencher( tupla ):
    palavra, id_texto = tupla
    if palavra.lower() not in dados_TF[id_texto]:
        dados_TF[id_texto][palavra.lower()] = 0
    dados_TF[id_texto][palavra.lower()] += 1
    try:
        indice_invertido [ palavra.lower() ].add(id_texto)
    except KeyError:
        indice_invertido[ palavra.lower() ] = set([ id_texto ])


Criação da função que cria o indice invertido. Percorre todos os textos e vai adicionando a indice invertido.

In [3]:
def extrair( texto , id_texto ):
    for i in xrange( len(texto) ):
        dados_TF[id_texto[i]]= {}
        for palavra in nltk.word_tokenize( str(texto[i]).decode('utf-8') ):
            preencher((palavra,id_texto[i]))
            

##Buscando dados
#MODELO TF
Função baseado no modelo de busca TF. Ele pega as palavras e da peso as documentos que possuem as determinadas plavras com a função de peso : docTF * qTF

In [4]:
def modeloTF(palavras):
    documentos = search(palavras)
    documentosTF = []
    lista = map( lambda x: x.decode('utf-8').strip(), palavras.split("AND") )
    for indice in documentos:
        documentoPeso = 0
        for palavra in lista:
            if (palavra in dados_TF[indice]):
                docTF = dados_TF[indice][palavra] 
            else:
                docTF = 0
            qTF = lista.count(palavra)
            documentoPeso += docTF * qTF
            
        documentosTF.append((indice, documentoPeso))
    documentosTF.sort(key=lambda tup: tup[1], reverse=True)
    indices = []
    for tupla in documentosTF:
        indices.append(tupla[0])
    return indices


#MODELO TF-IDF
Função baseado no modelo de busca TF-IDF. Ele pega as palavras e da peso as documentos que possuem as determinadas plavras com a função de peso : docTF * qTF * idf

In [5]:
def modeloIDF(palavras):
    documentos = search(palavras)
    documentosTF = []
    lista = map( lambda x: x.decode('utf-8').strip(), palavras.split("AND") )
    for indice in documentos:
        documentoPeso = 0
        for palavra in lista:
            if (palavra in dados_TF[indice]):
                docTF = dados_TF[indice][palavra] 
            else:
                docTF = 0
            qTF = lista.count(palavra)
            idf = math.log(file_csv.titulo.size/len(indice_invertido[ palavra.lower() ]))
            documentoPeso += docTF * qTF * idf
            
        documentosTF.append((indice, documentoPeso))
    documentosTF.sort(key=lambda tup: tup[1], reverse=True)
    indices = []
    for tupla in documentosTF:
        indices.append(tupla[0])
    return indices



#MODELO BINÁRIO
Função principal que procura no dicionário as palavras e retorna o id de todos os textos que possuem aquelas palavras.

In [6]:
def search( palavras ):
    if ( "AND" in palavras ):
        lista = map( lambda x: x.decode('utf-8').strip(), palavras.split("AND") )
        return procurarAND(lista)
    elif ("OR" in palavras ):
        lista = map(lambda x: x.decode('utf-8').strip(), palavras.split("OR"))
        return procurarOR(lista)
    else:
        return list(indice_invertido[palavras])



#MODELO BM25
Função baseado no modelo de busca BM25. Ele pega as palavras e da peso as documentos que possuem as determinadas plavras com a função de peso : ((docTF*(k+1))/(docTF+(k))) * qTF * idf, o valor de k escolhido foi 1.5.

In [7]:
def modeloBM25(palavras):
    documentos = search(palavras)
    documentosTF = []
    k=1.5
    lista = map( lambda x: x.decode('utf-8').strip(), palavras.split("AND") )
    for indice in documentos:
        documentoPeso = 0
        for palavra in lista:
            if (palavra in dados_TF[indice]):
                docTF = dados_TF[indice][palavra] 
            else:
                docTF = 0
            qTF = lista.count(palavra)
            idf = math.log(file_csv.titulo.size/len(indice_invertido[ palavra.lower() ]))
            documentoPeso += ((docTF*(k+1))/(docTF+k)) * qTF * idf
            
        documentosTF.append((indice, documentoPeso))
    documentosTF.sort(key=lambda tup: tup[1], reverse=True)
    indices = []
    for tupla in documentosTF:
        indices.append(tupla[0])
    return indices



Função auxiliar que procurar pelo OR e retorna o id de todos os textos que possuem pelo menos uma das palvras. OBS: Função genérica do OR.

In [8]:
def procurarOR(lista):
    indices = indice_invertido[lista[0].lower()]
    for palavra in lista:
        indices = set(reduce( lambda x,y : list( x | y ), [ indice_invertido[palavra.lower()] , indices] ))
    return indices

Função auxiliar que procurar pelo AND e retorna o id de todos os textos que possuem todas as palavras. OBS: Função genérica do AND.

In [9]:
def procurarAND(lista):
    indices = indice_invertido[lista[0].lower()]
    for palavra in lista:
        indices = set(reduce( lambda x,y : list( x & y ), [ indice_invertido[palavra.lower()] , indices] ))
    return indices

Função main que roda o programa.

In [10]:
if __name__ == "__main__":
    file_csv = pandas.read_csv("estadao_noticias_eleicao.csv")
    file_csv = file_csv.replace(np.nan, '', regex=True)
    texto = file_csv.titulo + " " + file_csv.conteudo + " " + file_csv.subTitulo
    extrair(texto,file_csv.idNoticia)
    

In [11]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])


Testando a aplicação¶

Função para testar alguns casos do sistema.


#TESTANDO A EFICIENCIA DOS MODELOS IMPLEMENTADOS

Podemos notar que os melhores foram Desempenho IDF e Desempenho BM25 e o pior foi Desempenho Binário com GOOGLE, isso já era de se esperar já que os modelos IDF E BM25 são aprimoramentos dos modelos anteriores, e o Binário é o modelo mais simples que tem. 

In [12]:
import ast

termos = "segundo AND turno"
termos1 =  "lava AND jato"
termos2 =  "projeto AND de AND lei"
termos3 = "compra AND de AND voto"
termos4 = "ministério AND público"

consultaBinaria = []
consultaTF = []
consultaIDF = []
consultaBM25 = []

consultaBinaria.append(list(search(termos))[:5])
consultaTF.append(modeloTF(termos)[:5])
consultaIDF.append(modeloIDF(termos)[:5])
consultaBM25.append(modeloBM25(termos)[:5])

consultaBinaria.append(list(search(termos1))[:5])
consultaTF.append(modeloTF(termos1)[:5])
consultaIDF.append(modeloIDF(termos1)[:5])
consultaBM25.append(modeloBM25(termos1)[:5])

consultaBinaria.append(list(search(termos2))[:5])
consultaTF.append(modeloTF(termos2)[:5])
consultaIDF.append(modeloIDF(termos2)[:5])
consultaBM25.append(modeloBM25(termos2)[:5])

consultaBinaria.append(list(search(termos3))[:5])
consultaTF.append(modeloTF(termos3)[:5])
consultaIDF.append(modeloIDF(termos3)[:5])
consultaBM25.append(modeloBM25(termos3)[:5])

consultaBinaria.append(list(search(termos4))[:5])
consultaTF.append(modeloTF(termos4)[:5])
consultaIDF.append(modeloIDF(termos4)[:5])
consultaBM25.append(modeloBM25(termos4)[:5])
    

gabarito = pandas.read_csv('gabarito.csv')

to_array = lambda text: ast.literal_eval(text)

# parsing string to array
gabarito.busca_binaria = gabarito.busca_binaria.apply(to_array)
gabarito.tf = gabarito.tf.apply(to_array)
gabarito.tfidf = gabarito.tfidf.apply(to_array)
gabarito.bm25 = gabarito.bm25.apply(to_array)
gabarito.google = gabarito.google.apply(to_array)



# create a dataframe with results
suaresposta = {'busca_binaria': consultaBinaria, 'tf': consultaTF,
              'tfidf': consultaIDF, 'bm25': consultaBM25}
suaresposta = pandas.DataFrame(data=suaresposta)


print("Precisão para consulta usando busca binaria %.2f%%" % (100 * mapk(gabarito.busca_binaria, suaresposta.busca_binaria, k=5)))

print("Precisão para consulta usando tf %.2f%%" % (100 * mapk(gabarito.tf,suaresposta.tf, k=5)))

print("Precisão para consulta usando tf_idf %.2f%%" % (100 * mapk(gabarito.tfidf, suaresposta.tfidf, k=5)))

print("Precisão para consulta usando bm25 %.2f%%" % (100 * mapk(gabarito.bm25, suaresposta.bm25, k=5)))

print

print("Precisão para consulta usando busca binaria %.2f%%" % (100 * mapk(gabarito.google, suaresposta.busca_binaria, k=5)))

print("Precisão para consulta usando tf em comparaçao com o google %.2f%%" % (100 * mapk(gabarito.google, suaresposta.tf, k=5)))

print("Precisão para consulta usando tf_idf em comparaçao com o google %.2f%%" % (100 * mapk(gabarito.google, suaresposta.tfidf, k=5)))

print("Precisão para consulta usando bm25 em comparaçao com o google %.2f%%" % (100 * mapk(gabarito.google, suaresposta.bm25, k=5)))

Precisão para consulta usando busca binaria 79.20%
Precisão para consulta usando tf 100.00%
Precisão para consulta usando tf_idf 86.87%
Precisão para consulta usando bm25 77.80%

Precisão para consulta usando busca binaria 0.00%
Precisão para consulta usando tf em comparaçao com o google 4.80%
Precisão para consulta usando tf_idf em comparaçao com o google 8.80%
Precisão para consulta usando bm25 em comparaçao com o google 16.80%
